In [14]:
import math
from math import sqrt
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import concat
from datetime import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
from pandas.core.algorithms import mode
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.python.keras.backend import concatenate
from tensorflow.python.keras.callbacks import History
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import LSTM,GRU,Bidirectional
from tensorflow.keras.layers import Dense, Dropout,Activation, Embedding, Permute,Input,Multiply,RepeatVector,Flatten
from keras.utils.vis_utils import plot_model

In [15]:
# 读取csv
df = pd.read_csv("data.csv",encoding='ISO-8859-1')
values=df.values #数值特征变量

In [ ]:
# 划分数据集
train_size = round(len(values) * 0.7)
train, test = values[0:train_size], values[train_size:len(values)]
print(train.shape)
print(test.shape)

In [17]:
# 归一化
values = values.astype('float32') #使得所有数值类型都是float类型
scaler = MinMaxScaler(feature_range = (0, 1)) #0-1归一化
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.fit_transform(test)

In [ ]:
# 定义输入和输出
def createXY(dataset,n_past):
    data_X = []
    data_Y = []
    for i in range(n_past, len(dataset)):
        data_X.append(dataset[i - n_past:i, 0:dataset.shape[1]])
        data_Y.append(dataset[i,0])
    return np.array(data_X),np.array(data_Y)
train_X,train_Y=createXY(train_scaled,9)
test_X,test_Y=createXY(test_scaled,9)
print("train_X Shape--",train_X.shape)
print("train_Y Shape--",train_Y.shape)
print("test_X Shape--",test_X.shape)
print("test_Y Shape--",test_Y.shape)

In [ ]:
model_input = Input(shape=(train_X.shape[1],train_X.shape[2]))
x = LSTM(256)(model_input)
x = Dropout(0.4)(x)
x = GRU(256)(x)
x = Dropout(0.3)(x)
x = Dense(1)(x)
model = Model(model_input, x)
adam=tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mse', optimizer=adam)
model.summary()

In [ ]:
history = model.fit(train_X, train_Y, epochs=100, batch_size=77, validation_data=(test_X, test_Y), verbose=2, shuffle=True)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
#评估模型效果
test_X=test_X.reshape((test_X.shape[0],test_X.shape[1]*test_X.shape[2]))
inv_yhat=concatenate((yhat, test_X[:, 0:10]), axis=1)
inv_yhat=scaler.inverse_transform(inv_yhat)
inv_yhat=inv_yhat[:,0]
test_Y=test_Y[:,np.newaxis]
inv_y=concatenate((test_Y, test_X[:, 0:10]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
mae=(mean_absolute_error(inv_y, inv_yhat))
r2=(r2_score(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)
print('Test MAE: %.3f' % mae)
print('Test R2: %.3f' % r2)

In [ ]:
train_X=train_X.reshape((train_X.shape[0],train_X.shape[1]*train_X.shape[2]))
inv_yhat1=concatenate((yhat1, train_X[:, 0:10]), axis=1)
inv_yhat1=scaler.inverse_transform(inv_yhat1)
inv_yhat1=inv_yhat1[:,0]
train_Y=train_Y[:,np.newaxis]
inv_y1=concatenate((train_Y, train_X[:, 0:10]), axis=1)
inv_y1 = scaler.inverse_transform(inv_y1)
inv_y1 = inv_y1[:,0]
RealValue=concatenate((inv_y1,inv_y),axis=0)
PredictValue=concatenate((inv_yhat1,inv_yhat),axis=0)

In [ ]:
f, (ax1) = plt.subplots(figsize=(6, 4), nrows=1)
ax1.plot(RealValue,linewidth=2,label="Real Value")
ax1.plot(PredictValue,  linewidth=2,label="Predict Value")
plt.legend()